In [ ]:
#@markdown <center><h1>Install</h1></center>

%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

flux_version = "dev" # @param ["dev","schnell"]

if flux_version == "schnell":
  !aria2c --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/flux1-schnell.safetensors -d /content/TotoroUI/models/unet -o flux1.safetensors
elif flux_version == "dev":
  !aria2c --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1.safetensors

!aria2c --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro_extras import nodes_flux
from totoro import model_management
from google.colab import files

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

    unet_f, clip_f = unet, clip

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

In [ ]:
#@markdown <center><h1>Load/Unload Lora</h1></center>

loras = {
    "xlabs_flux_anime":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/anime_lora_comfy_converted.safetensors",
         "filename": "xlabs_anime_lora.safetensors",
         "triggers": "anime"
         },
    "xlabs_flux_art":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/art_lora_comfy_converted.safetensors",
         "filename": "xlabs_art_lora.safetensors",
         "triggers": "art"
         },
    "xlabs_flux_disney":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/disney_lora_comfy_converted.safetensors",
         "filename": "xlabs_disney_lora.safetensors",
         "triggers": "disney style"
         },
    "xlabs_flux_mjv6_lora":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/mjv6_lora_comfy_converted.safetensors",
         "filename": "xlabs_mjv6_lora.safetensors",
         "triggers": "none"
         },
    "xlabs_flux_realism":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/realism_lora_comfy_converted.safetensors",
         "filename": "xlabs_realism_lora.safetensors",
         "triggers": "none"
         },
    "xlabs_flux_scenery":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/scenery_lora_comfy_converted.safetensors",
         "filename": "xlabs_scenery_lora.safetensors",
         "triggers": "scenery style"
         },
    "xlabs_flux_furry":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/furry_lora.safetensors",
         "filename": "xlabs_flux_furry_lora.safetensors",
         },
}

lora_name = "no_lora" # @param ["no_lora", "xlabs_flux_anime", "xlabs_flux_art", "xlabs_flux_disney", "xlabs_flux_mjv6_lora", "xlabs_flux_realism", "xlabs_flux_scenery"]
lora_strength_model = 1 # @param {"type":"slider","min":0,"max":1,"step":0.1}
lora_strength_clip = 1 # @param {"type":"slider","min":0,"max":1,"step":0.1}

if lora_name == "no_lora":
  unet_f, clip_f = unet, clip

  print("Lora Unloaded")
else:
  lora = loras.get(lora_name, None)

  if lora:
    !aria2c --download-result=hide --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M {lora["url"]} -d /content/TotoroUI/models/loras -o {lora["filename"]}

    unet_f, clip_f = LoraLoader.load_lora(unet, clip, lora["filename"], lora_strength_model, lora_strength_clip)

    print(f"Lora Loaded: {lora['filename']}")
    print(f"Triggers: {lora['triggers']}")
  else:
    print("Lora is not listed")

In [ ]:
#@markdown <center><h1>Txt2Img</h1></center>

with torch.inference_mode():
    positive_prompt = "black forest toast spelling out the words 'FLUX DEV', tasty, food photography, dynamic shot" # @param {"type":"string"}
    width = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
    height = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
    fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
    guidance = 3.5 # @param {"type":"slider","min":0,"max":20,"step":0.5}
    steps = 20 # @param {"type":"slider","min":4,"max":50,"step":1}
    sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
    scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
    batch_size = 1 # @param {"type":"slider","min":1,"max":20,"step":1}
    auto_download = False # @param {"type":"boolean"}

    cond, pooled = clip_f.encode_from_tokens(clip_f.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]
    guider = BasicGuider.get_guider(unet_f, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_f, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]

    for i in range(0, batch_size):
      if fixed_seed == 0:
        seed = random.randint(0, 18446744073709551615)
      else:
        seed = fixed_seed

      print("Seed:", seed)

      noise = RandomNoise.get_noise(seed)[0]
      sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
      model_management.soft_empty_cache()
      decoded = VAEDecode.decode(vae, sample)[0].detach()
      Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/flux_t2i_{seed}_{i}.png")

      if auto_download:
        files.download(f"/content/flux_t2i_{seed}_{i}.png")

      img = Image.open(f"/content/flux_t2i_{seed}_{i}.png")
      display(img)

In [ ]:
#@markdown <center><h1>Img2Img</h1></center>

with torch.inference_mode():
    positive_prompt = "anime style" # @param {"type":"string"}
    width = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
    height = 1024 # @param {"type":"slider","min":256,"max":2048,"step":1}
    fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
    guidance = 3.5 # @param {"type":"slider","min":0,"max":20,"step":0.5}
    steps = 20 # @param {"type":"slider","min":4,"max":50,"step":1}
    sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
    scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
    input_img = "/content/test.png" # @param {"type":"string"}
    denoise = 0.85 # @param {"type":"slider","min":0,"max":1,"step":0.01}
    auto_download = False # @param {"type":"boolean"}

    if fixed_seed == 0:
      seed = random.randint(0, 18446744073709551615)
    else:
      seed = fixed_seed

    print("Seed:", seed)

    cond, pooled = clip_f.encode_from_tokens(clip_f.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_f, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_f, scheduler, steps, denoise)[0]

    image = nodes.LoadImage().load_image(input_img)[0]
    latent_image = ImageScaleToTotalPixels.upscale(image, "lanczos", 1.0)[0]
    latent_image = VAEEncode.encode(vae, latent_image)[0]

    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/flux_i2i_{seed}.png")

if auto_download:
  files.download(f"/content/flux_i2i_{seed}.png")

img = Image.open(f"/content/flux_i2i_{seed}.png")
display(img)